In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import time
import torch
from torch.optim import Adam
from torch.nn import MSELoss
from torch import nn
from torch.utils.data import DataLoader, Dataset

In [22]:
NEURAL_NETWORK_EPOCHS = [10, 50, 100, 200, 500]

In [23]:
data = pd.read_csv("./data/concrete_strength/Concrete_Data_Yeh.csv")

In [24]:
X = data.drop("csMPa", axis=1)
y = data["csMPa"]

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [26]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [27]:
def calculate_running_time(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        model = func(*args)
        end = time.time() - start
        print(f"Function runned in {end} seconds.")
        return model
    return wrapper

In [28]:
@calculate_running_time
def train_forest(X, y):
    model = RandomForestRegressor()
    model.fit(X, y)
    return model

In [29]:
class DeepLearningModel(nn.Module):
    def __init__(self, activation_function):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(8, 16),
            activation_function(),
            nn.Linear(16, 32),
            activation_function(),
            nn.Linear(32, 64),
            activation_function(),
            nn.Linear(64, 32),
            activation_function(),
            nn.Linear(32, 16),
            activation_function(),
            nn.Linear(16, 8),
            activation_function(),
            nn.Linear(8, 1),
        )
        
        
    def forward(self, X):
        return self.layers(X)

In [30]:
class DataProvider(Dataset):
    def __init__(self, X, y):
        super().__init__()

        self.X = torch.tensor(X).float()
        self.y = torch.tensor(y.values).float()
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [31]:
def build_model(activation_function):
    return DeepLearningModel(activation_function)

In [32]:
def train_model_for_num_of_epochs(epochs: int, trained_models: dict, data: DataLoader):
    time_start = time.time()
    
    model_name = f"Model-{epochs}"
    loss_func = MSELoss()
    model = build_model(nn.ReLU)
    optimizer = Adam(model.parameters(), lr=0.0001)
    
    for epoch in range(epochs):
        for inp, out in data:
            optimizer.zero_grad()
            out = torch.unsqueeze(out, -1)
            prediction = model(inp)
            loss = loss_func(prediction, out)
            loss.backward()
            optimizer.step()
            
        
    print(f"Model name {model_name}: Loss: {loss.item()}")
    
    trained_models[model_name] = {
        "Time": (time.time() - time_start),
        "Epochs": epochs,
        "Model": model,
        "Optimizer": optimizer,
        "Loss function": loss_func
    }

In [33]:
def get_results(data: DataLoader, model: DeepLearningModel):
    loss_func = MSELoss()
    data_len = len(data)
    mse_sum = 0
    
    for inp, out in data:
        out = torch.unsqueeze(out, -1)
        preds = model(inp)
        loss = loss_func(preds, out)
        mse_sum += loss.item()
    
    mse = mse_sum / data_len
    print(mse)
        

In [34]:
train_provider = DataProvider(X_train, y_train)
train_iterator = DataLoader(train_provider, batch_size=32, shuffle=False)

In [35]:
test_provider = DataProvider(X_test, y_test)
test_iterator = DataLoader(test_provider, batch_size=32, shuffle=False)

In [36]:
forest = train_forest(X_train, y_train)

Function runned in 0.6383416652679443 seconds.


In [37]:
forest_predictions = forest.predict(X_test)
forest_results = mean_squared_error(y_test, forest_predictions)
forest_results

31.294733605492397

In [38]:
models = {}

In [39]:
for num_of_epochs in NEURAL_NETWORK_EPOCHS:
    train_model_for_num_of_epochs(num_of_epochs, models, train_iterator)

for num_of_epochs in NEURAL_NETWORK_EPOCHS:
    get_results(test_iterator, models[f"Model-{num_of_epochs}"]["Model"])

Model name Model-10: Loss: 864.9705200195312
Model name Model-50: Loss: 81.90930938720703
Model name Model-100: Loss: 72.11370849609375
Model name Model-200: Loss: 47.4844856262207
Model name Model-500: Loss: 10.591087341308594
1543.8618842230903
165.45723130967883
153.00692452324762
79.56938552856445
47.30097368028429


In [40]:
for num_of_epochs in NEURAL_NETWORK_EPOCHS:
    print(models[f"Model-{num_of_epochs}"]["Time"])

0.5589962005615234
2.67238712310791
5.355464458465576
10.854358434677124
27.134477138519287
